### Dataset utilisé: 

```python
# Load the dataset
data = np.load('ckn_mnist.npz')
# Access the arrays in the dataset
images = data['X']
labels = data['y']
#  Binarization of the labels
labels = np.where(labels < 5, -1, 1)
# Take n samples
n=1000
np.random.seed(0)
x_values = np.random.choice(images.shape[0] - n, n, replace=False)
images_ = images[x_values]
labels_ = labels[x_values]
# Split the dataset into a training set and a test set
n_train = int(n*0.7)
train_images = images_[:n_train]
train_labels = labels_[:n_train]
test_images = images_[n_train:]
test_labels = labels_[n_train:]
# Reshape the images and the labels
X_train = train_images.reshape(train_images.shape[0], -1)
X_test = test_images.reshape(test_images.shape[0], -1)
y_train = train_labels.reshape(-1)
y_test = test_labels.reshape(-1)
```

---

### Fichiers utilisés dans le dossier Data:
1. **targets_list.pkl** : contient la liste des "targets norm".
2. **learning_rates_list.pkl** : contient la liste des "learning rates".
3. **Dynamique/alpha_list.pkl** : Liste des itérés pour différentes "targets norm" avec *dynamic stepsize*.
4. **Dossier GD** : comporte 20 fichiers **learning_rate_xx.xxxxx.pkl** : Liste des itérés pour différentes "targets norm" et pour le "learning rate xx.xxxxx" pour *gradient descent*.


---
#### Utilisation pour Dynamic stepsize :

```python
with open('Data/dynamique/alpha_lists.pkl', 'rb') as f:
    with open('Data/targets_list.pkl', 'rb') as f1:
        targets = pickle.load(f1)
        loaded_alpha_lists = pickle.load(f)
        for i in range(len(targets)):
            alpha_list = loaded_alpha_lists[i]
            target = targets[i]# Liste des itérés pour la target norm 'target' 
```

#### utilisation pour Gradient descent:       

```python
with open('Data/learning_rates_list.pkl', 'rb') as f1:
    learning_rates = pickle.load(f1)

with open('Data/targets_list.pkl', 'rb') as f2:
    targets = pickle.load(f2)

for learning_rate in learning_rates:
    with open(r'Data/GD/learning_rate_{:.5f}.pkl'.format(learning_rate), 'rb') as f:
        loaded_alpha_lists = pickle.load(f)
        for i in range(len(targets)):
            target = targets[i]
            alpha_list = loaded_alpha_lists[i] # liste des itérés pour la target norm 'target' et le leraning rate 'learning_rate'
```

In [14]:
from Modele import Modele
from Optimiseur import Optimiseur
import numpy as np
import pickle


In [ ]:
# Load the dataset
data = np.load('ckn_mnist.npz')
# Access the arrays in the dataset
images = data['X']
labels = data['y']
#  Binarization of the labels
labels = np.where(labels < 5, -1, 1)
# Take n samples
n=1000
np.random.seed(0)
x_values = np.random.choice(images.shape[0] - n, n, replace=False)
images_ = images[x_values]
labels_ = labels[x_values]
# Split the dataset into a training set and a test set
n_train = int(n*0.7)
train_images = images_[:n_train]
train_labels = labels_[:n_train]
test_images = images_[n_train:]
test_labels = labels_[n_train:]
# Reshape the images and the labels
X_train = train_images.reshape(train_images.shape[0], -1)
X_test = test_images.reshape(test_images.shape[0], -1)
y_train = train_labels.reshape(-1)
y_test = test_labels.reshape(-1)

# Randoms initializations for the alpha
init = np.random.randn(X_train.shape[0])

# Define parameters and create Modele and Optimiseur objects
modele = Modele(0,30)
optimiseur = Optimiseur(modele)
max_iters=5000
targets = np.linspace(1e-4,1e-2,20)
criterion='norm'
L,mu = modele.constante_L(X_train)
learning_rates = np.linspace(1/L,1.99/L,20)

for learning_rate in learning_rates:
    alpha_lists = []
    for target in targets:
        # Run the optimisation for one target 
        alpha, alpha_list = optimiseur.gradient_descent(X_train, y_train, init.copy(),learning_rate, max_iters, target, criterion)
        alpha_list = np.array(alpha_list)
        alpha_lists.append(alpha_list)

    # Ensure the directory exists
    import os
    os.makedirs('Data/GD', exist_ok=True)
    
    # Save the alpha_lists to a file
    with open(r'Data/GD/learning_rate_{:.5f}.pkl'.format(learning_rate), 'wb') as f:
        pickle.dump(alpha_lists, f)


In [16]:
with open('Data/targets_list.pkl', 'wb') as f:
    pickle.dump(targets, f)
with open('Data/learning_rates_list.pkl', 'wb') as f:
    pickle.dump(learning_rates, f)

In [17]:
alpha_lists = []
for target in targets:
    # Run the optimisation for one target
    alpha, alpha_list = optimiseur.dynamic_stepsize(X_train, y_train, init.copy(), max_iters, target, criterion)
    alpha_list = np.array(alpha_list)
    alpha_lists.append(alpha_list)

# Ensure the directory exists
import os
os.makedirs('Data/Dynamique', exist_ok=True)

# Save the alpha_lists to a file
with open('Data/Dynamique/alpha_lists.pkl', 'wb') as f:
    pickle.dump(alpha_lists, f)


In [19]:
with open('Data/learning_rates_list.pkl', 'rb') as f1:
    learning_rates = pickle.load(f1)

with open('Data/targets_list.pkl', 'rb') as f2:
    targets = pickle.load(f2)

for learning_rate in learning_rates:
    with open(r'Data/GD/learning_rate_{:.5f}.pkl'.format(learning_rate), 'rb') as f:
        loaded_alpha_lists = pickle.load(f)
        for i in range(len(targets)):
            target = targets[i]
            alpha_list = loaded_alpha_lists[i]
            print("target norm: ", target)
            print("learning rate: ", learning_rate)
            print("accuracy: ", modele.accuracy(X_train,X_test, y_test, alpha_list[-1]))
            print("norm: ", np.linalg.norm(alpha_list[-1]))
            print("\n")

target norm:  0.0001
learning rate:  31.543420070383792
accuracy:  0.9633333333333334
norm:  16.828978868898087


target norm:  0.0006210526315789474
learning rate:  31.543420070383792
accuracy:  0.9633333333333334
norm:  16.856255643114135


target norm:  0.0011421052631578948
learning rate:  31.543420070383792
accuracy:  0.9633333333333334
norm:  16.87067298151948


target norm:  0.0016631578947368423
learning rate:  31.543420070383792
accuracy:  0.9666666666666667
norm:  16.876865846110977


target norm:  0.0021842105263157894
learning rate:  31.543420070383792
accuracy:  0.9666666666666667
norm:  16.879821625220636


target norm:  0.0027052631578947366
learning rate:  31.543420070383792
accuracy:  0.9666666666666667
norm:  16.882281500728737


target norm:  0.0032263157894736843
learning rate:  31.543420070383792
accuracy:  0.9666666666666667
norm:  16.88583130790356


target norm:  0.0037473684210526316
learning rate:  31.543420070383792
accuracy:  0.97
norm:  16.89031566047413




In [18]:
with open('Data/dynamique/alpha_lists.pkl', 'rb') as f:
    with open('Data/targets_list.pkl', 'rb') as f1:
        targets = pickle.load(f1)
        loaded_alpha_lists = pickle.load(f)
        for i in range(len(targets)):
            alpha_list = loaded_alpha_lists[i]
            target = targets[i]
            print('target norm: ', target)
            print("accuracy: ", modele.accuracy(X_train,X_test, y_test, alpha_list[-1]))
            print("norm: ", np.linalg.norm(alpha_list[-1]))
            print("\n")

target norm:  0.0001
accuracy:  0.9633333333333334
norm:  16.828978868898087


target norm:  0.0006210526315789474
accuracy:  0.9633333333333334
norm:  16.856255643114135


target norm:  0.0011421052631578948
accuracy:  0.9633333333333334
norm:  16.87067298151948


target norm:  0.0016631578947368423
accuracy:  0.9666666666666667
norm:  16.876865846110977


target norm:  0.0021842105263157894
accuracy:  0.9666666666666667
norm:  16.879821625220636


target norm:  0.0027052631578947366
accuracy:  0.9666666666666667
norm:  16.882281500728737


target norm:  0.0032263157894736843
accuracy:  0.9666666666666667
norm:  16.88583130790356


target norm:  0.0037473684210526316
accuracy:  0.97
norm:  16.89031566047413


target norm:  0.004268421052631579
accuracy:  0.97
norm:  16.903248154099344


target norm:  0.004789473684210527
accuracy:  0.97
norm:  16.909950433207783


target norm:  0.005310526315789474
accuracy:  0.97
norm:  16.929583772656798


target norm:  0.005831578947368421
accuracy